## Assignment Part-1 Week 3

Using beautifulsoup to scrape the table in wikipedia page

In [1]:
!pip install bs4

     |████████████████████████████████| 122kB 9.3MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4


In [2]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from urllib.request import urlopen
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = urlopen(url) 
soup = BeautifulSoup(html, 'html.parser')

## Getting the tables present in the page

In [3]:
table = soup.find_all('table')

## Retreiving the columns from the table

In [4]:
postal_code = []
borough = []
neighbourhood = []

rows = table[0].find_all('tr')
    
for row in rows:
    cells = row.find_all('td')
     
        
    if len(cells) > 1:
        code = cells[0]
        postal_code.append(code.text.strip())
            
        b = cells[1]
        borough.append(b.text.strip())
            
        neighbour = cells[2]
        neighbourhood.append(neighbour.text.strip())
            

## Creating a dataframe from the columns

In [5]:
data = pd.DataFrame([postal_code,borough,neighbourhood])
df = data.T
df.columns = ['postal_code','borough','neighbourhood']
df.head()

,postal_code,borough,neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [6]:
df.shape

(180, 3)

## Dropping the rows where borough is Not assigned.

In [7]:
df = df[df['borough']!='Not assigned']
df.head()

,postal_code,borough,neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## Assigning the Borough values to rows where Neighbourhood is Not Assigned

In [8]:
df.loc[df['neighbourhood'] == 'Not assigned', 'neighbourhood'] = df['borough']
df.head()

,postal_code,borough,neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## Getting dataframe shape

In [9]:
df.shape

(103, 3)

## Load latitude and longitude data

In [10]:
data = pd.read_csv('https://cocl.us/Geospatial_data')

In [11]:
data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
df = df.rename(columns={"postal_code": "Postal Code"}).reset_index()
df = df.drop(columns='index')
df.head()

,Postal Code,borough,neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## Merging the two data

In [13]:
merged_data = pd.merge(df, data, on='Postal Code', how='outer')
merged_data.head()

,Postal Code,borough,neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
